In [1]:
import numpy as np

class FastVector1:
    """
    Minimal wrapper for fastvector embeddings.
    ```
    Usage:
        $ model = FastVector(vector_file='/path/to/wiki.en.vec')
        $ 'apple' in model
        > TRUE
        $ model['apple'].shape
        > (300,)
    ```
    """

    def __init__(self, vector_file='', transform=None):
        """Read in word vectors in fasttext format"""
        self.word2id = {}

        # Captures word order, for export() and translate methods
        self.id2word = []

        print('reading word vectors from %s' % vector_file)
        with open(vector_file, 'r') as f:
	    print ('1') 
            (self.n_words, self.n_dim) = \
            (int(x) for x in f.readline().rstrip('\n').split(' '))
            self.embed = np.zeros((self.n_words, self.n_dim))
            for i, line in enumerate(f):
                elems = line.rstrip('\n').split(' ')
                self.word2id[elems[0]] = i
                #print (elems[0])
                self.embed[i] = elems[1:self.n_dim+1]
                self.id2word.append(elems[0])
        
        # Used in translate_inverted_softmax()
        self.softmax_denominators = None
        
        if transform is not None:
            print('Applying transformation to embedding')
            self.apply_transform(transform)
    
    def apply_cop(self, matrix,i):
        self.embed[i]=matrix[:]
    
    def export(self, outpath):
        """
        Transforming a large matrix of WordVectors is expensive. 
        This method lets you write the transformed matrix back to a file for future use
        :param The path to the output file to be written 
        """
        fout = open(outpath, "w")

        # Header takes the guesswork out of loading by recording how many lines, vector dims
        fout.write(str(self.n_words) + " " + str(self.n_dim) + "\n")
        for token in self.id2word:
            vector_components = ["%.6f" % number for number in self[token]]
            vector_as_string = " ".join(vector_components)

            out_line = token + " " + vector_as_string + "\n"
            fout.write(out_line)

        fout.close()
    
    
    @classmethod
    
    def __contains__(self, key):
        return key in self.word2id

    def __getitem__(self, key):
        return self.embed[self.word2id[key]]

In [2]:
class FastVector2:
    """
    Minimal wrapper for fastvector embeddings.
    ```
    Usage:
        $ model = FastVector(vector_file='/path/to/wiki.en.vec')
        $ 'apple' in model
        > TRUE
        $ model['apple'].shape
        > (300,)
    ```
    """

    def __init__(self, vector_file='', transform=None):
        """Read in word vectors in fasttext format"""
        self.word2id = {}

        # Captures word order, for export() and translate methods
        self.id2word = []

        print('reading word vectors from %s' % vector_file)
        with open(vector_file, 'r') as f:
	    print ('1') 
            (self.n_words, self.n_dim) = \
            (int(x) for x in f.readline().rstrip('\n').split(' '))
            self.embed = np.zeros((self.n_words, self.n_dim))
            for i, line in enumerate(f):
                elems = line.rstrip('\n').split(' ')
                self.word2id[elems[0]] = i
                #print (elems[0])
                ini=np.random.rand(300)
                self.embed[i] = ini[:] ##elems[1:self.n_dim+1]
                self.id2word.append(elems[0])
        
        # Used in translate_inverted_softmax()
        self.softmax_denominators = None
        
        if transform is not None:
            print('Applying transformation to embedding')
            self.apply_transform(transform)
    
    def apply_cop(self, matrix,i):
        self.embed[i]=matrix[:]
    
    def export(self, outpath):
        """
        Transforming a large matrix of WordVectors is expensive. 
        This method lets you write the transformed matrix back to a file for future use
        :param The path to the output file to be written 
        """
        fout = open(outpath, "w")

        # Header takes the guesswork out of loading by recording how many lines, vector dims
        fout.write(str(self.n_words) + " " + str(self.n_dim) + "\n")
        for token in self.id2word:
            vector_components = ["%.6f" % number for number in self[token]]
            vector_as_string = " ".join(vector_components)

            out_line = token + " " + vector_as_string + "\n"
            fout.write(out_line)

        fout.close()
    
    
    @classmethod
    
    def __contains__(self, key):
        return key in self.word2id

    def __getitem__(self, key):
        return self.embed[self.word2id[key]]

In [3]:
def cosine_similarity(vec_a, vec_b):
        """Compute cosine similarity between vec_a and vec_b"""
        return np.dot(vec_a, vec_b) / \
            (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


In [4]:
import numpy as np

In [5]:

def levenshteinDistance(s1, s2):
    char='@'
    dif=0
    if len(s1) > len(s2):
        s1, s2 = s2, s1
        #print s1,s2
    distances = range(len(s1) + 1)
    #print distances
    #print enumerate(s2)
    for i2, c2 in enumerate(s2):
        #print i2,c2
        distances_ = [i2+1]
        #print distances_
        for i1, c1 in enumerate(s1):
            #print distances_,i1, c1, c2
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
        #print distances, char
        if 0 not in distances and char=='@':
            char=c2

        if char in ['\'','k','g','j']:
            dif=1
        #print 'hi', distances[-1], char
        #print char
    return distances[-1],dif

In [12]:
def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []
    ti=[]
    count=0
    for (source, target) in bilingual_dictionary:
        #print source,target
        if source in source_dictionary.word2id and target in target_dictionary.word2id:
            #print source, target
            count=count+1
            #print source, target
            ti.append(target_dictionary.word2id[target])
            source_matrix.append(source_dictionary[source])
            target_matrix.append(target_dictionary[target])
        if source in source_dictionary.word2id and target not in target_dictionary.word2id:
            for q in target_dictionary.word2id:
                dis,dif=levenshteinDistance(target,q)
                if dis==1 and dif==1:
                    count=count+1
                    print source, target,q
                    ti.append(target_dictionary.word2id[q])
                    source_matrix.append(source_dictionary[source])
                    target_matrix.append(target_dictionary[q])
                    #count+=1
                    #print p, q
    # return training matrices
    print count
    return np.array(source_matrix), np.array(target_matrix), np.array(ti)

In [7]:
en_dictionary = FastVector1(vector_file='/home/apatra/fastText/fastText_multilingual-master/eng.vec')
mi_dictionary = FastVector2(vector_file='/home/apatra/fastText/fastText_multilingual-master/model.vec')

en_vector = en_dictionary["one"]
mi_vector = mi_dictionary["newt"]
print(cosine_similarity(en_vector, mi_vector))

reading word vectors from /home/apatra/fastText/fastText_multilingual-master/eng.vec
1
reading word vectors from /home/apatra/fastText/fastText_multilingual-master/model.vec
1
0.013999010745350704


In [8]:
mi_words = set(mi_dictionary.word2id.keys())
en_words = set(en_dictionary.word2id.keys())

In [9]:
import codecs
bilingual_dictionary=[]
with codecs.open('/home/apatra/fastText/fastText_multilingual-master/eng-mic','r','utf-8') as f:
    for line in f:
        eng, mic=line.split(', ')
        #print eng
        eng=eng.strip('\"')
        #print eng
        mic=mic.strip('\"')
        mic=mic.replace('\n','')
        mic=mic.replace('"','')
        #print eng, mic
        bilingual_dictionary.append((eng,mic))
#print bilingual_dictionary

In [13]:
# form the training matrices
#from copy import deepcopy
source_matrix, target_matrix ,ti= make_training_matrices(
    en_dictionary, mi_dictionary, bilingual_dictionary)
#print len(source_matrix), len(target_matrix)
# learn and apply the transformation
#print ti
#target_matrix=deepcopy(source_matrix)
#print source_matrix[60][9], target_matrix[60][9]
#transform = learn_transformation(source_matrix, target_matrix)
#print type(transform)
#print transform[299]
#en_dictionary.apply_transform(transform)

/home/apatra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


all msʼt ms't
German alman klman
good geluʼlg gelu'lg
Micmac Miʼkmaq Mi'kmaq
Micmac Miʼkmaq Mikkmaq
abandon naqtʼg naqt'g
ache gesnugwaj gesnugwa
add mawgitg mawgi'g
alive mimajig mimaji
angel ansaleʼwit ansale'wit
anyway meʼ me'
anyway meʼ mej
arrive igaʼq iga'q
aspen miti mitji
ay eʼe eke
aye eʼe eke
babysit nespit kespit
beaver gopit opit
beaver gopit kopit
bee amu kmu
bee amu amuj
believe getlamsʼtuatl getlams'tuatl
believe getlamsʼtg getlams'tg
between miaweʼg miawe'g
big mesgiʼg mesgi'g
big mesgilg mesgi'g
bird sisip sisipk
blabbermouth seitun geitun
blacksmith glaptan klaptan
blessed sapeʼwit sape'wit
blessed sapeʼwig sape'wig
blocked gepjoqteg gep'joqteg
blood maltew mal'tew
bloody maltewig maltewi
born ugsʼtqamuit ugs'tqamuit
boss saqamawit saqamawik
boy lpaʼtuj lpa'tuj
bug jujij juji
bug jujij ujij
build eliatl ejiatl
butcher neʼpatat ne'patat
can gisaʼtoq gisa'toq
canoe gwitn kwitn
catch mesnʼg mesn'g
catch mesnatl megnatl
chief saqamawit saqamawik
Christ Seʼsus Se'sus
Chris

In [14]:
from collections import defaultdict

def list_duplicates(seq, x):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return (locs for key,locs in tally.items() 
            if key==x)

'''
ind=6
p=list_duplicates(ti,ti[ind])
#for l in p:
 #   print l
j=np.zeros(300)
print source_matrix[ind]
print source_matrix[191]
for l in p:
    for x in l:
        j+=source_matrix[x]
            
    target_matrix[ind]=j[:]/len(l)
print target_matrix[ind]
'''

'\nind=6\np=list_duplicates(ti,ti[ind])\n#for l in p:\n #   print l\nj=np.zeros(300)\nprint source_matrix[ind]\nprint source_matrix[191]\nfor l in p:\n    for x in l:\n        j+=source_matrix[x]\n            \n    target_matrix[ind]=j[:]/len(l)\nprint target_matrix[ind]\n'

In [15]:
import copy
j=np.zeros(300)
for r in range(0,len(ti)):
    #print source_matrix[r], target_matrix[r]
    #print len(source_matrix[r]),len(target_matrix[r])
    p=list_duplicates(ti,ti[r])
    j=np.zeros(300)
    for l in p:
        for x in l:
            j+=source_matrix[x]
            
        target_matrix[r]=j[:]/len(l)
    #target_matrix[r]=source_matrix[r][:]
    mi_dictionary.apply_cop(target_matrix[r],ti[r])

In [16]:
for tar in mi_dictionary.word2id:
    print tar


kisi-mawanqa’tumkek
aklasiewaloqsultiji
esma’titl
gelulatl
etlitum
Malpalewitelemut
gi'lewei
Western
litaqnwikas’kl
mimajuinuigtug
Valle
atkitemi
mikwite’tmu
maw-wlamu’k
Kaplno'l
kiwto’qa’sikewe’l
elakutultijik
apgwa'tuan
jijgluewjig
iknemat
pma'toq
nemiatijel
wels'tua'tigul
kajjimenewa’kik
wtuisunmual
iknemaj
wuji
wejgwa'lawoqo
teke’k
nasgua'tiji
nuj-seknika’tijik)
telkisi
mawelkisni
a'tugwaqa
tetpoqpilaqn
Naqtiki’l
pem-ajelkik
westawu’lkw
pajjoqe’kemk
kidi
kisikuitl
ukwita'q
apu’ksinew
seskutaq
la'lukete
wije’wmi’ti
ta'n
un'jan
gji'jiaqat
nutaptukl
Vernon
(rattle)
Mawiomijijel
Eleke’waki
ntawa’qa’taqatijik
pemi______________elkomiktule
nutankuat
Melkiknaq
te'sukunit
Snapshot
ktantekelin
kmu’ji’jek
mime’jk
wape'gna'milij
toqi-lukuti’tiji
Morrel
eliaql
wejkwita’jik
Lt
pitewey
keknuika
gisgaja'tuloqol
pemkaqieji
wi'kik
aqqtetpapo’tasin
kis-tl-teluaq
iknmutl
eliewap
tel-mil-lukutiek
tewijo’ltisnik
poqtukwatmi’tis
eluéultiek
sapa’lukwekaq
walipo
Npu'kwi'te'lmanen
Hil
Him
piskwikiaqel
wutm

ta'pu
tluej
tluei
tluen
nisa’sik
ksinukwas
“Pun
wikuwa
tela'tegeiap
newtiskaq
etl-na’taqamita’m
newtipuna'likl
kejitu’nl
teli-kjijituekip
nemitutis
a’maq
teli-i’k
Ekwitamtite’wk
kikmanu
samqwanji'j
etl-inkla'titaq
Ni’n
nukumijinen
kwesawe’k
Kmitkína
Maliamj
tpitelems
elulasnl
wiji-panutmi’tip
ewi’kika’tiji
wuji’jl
Telte’t
ngute'jitaq
etlankua
“Tlia
pessimkewe’
Maw-kejika
wilu’e
telwi'kas'k
Ntilsutaqanm
aknimiskisni’
tewo’tum
Chedabuctu
matenaqewaqenn
Bernard
alamiaqnewey-iktu
tkey
Nuji-kina’masultitewk
maliaptmi’tij
apankitatim
Samaliaewa'gig
mawa'matultine
maliaptmi’tis
mkite’taqney
iknemuetasinuk
Pekwatekemk
nuji-apoqnmua'tijik
Maljewejk
pa’sek
pejo'tu'titl
tekweywiekip
ut-utanmua
ansama
'gsalata
teljugu'e
wksitawi-ewanik
metui
ut-ektlamsetaqeneymuo
etl-lukwatmi’tij
Tua’luj
gsal
ne’pataqatijikNenaqite’tmi’tij
Peji-mittukwalkip
l’nu’
ut-lukewaqen
MacDonaldo’
'gji'nmu
putu
tlimastesk
ligutinew
wunaqo'tasik
emeteskultio
nikwenawemk
Flor
Guernse
nklamuksi
'gji'tutew
Nike'alme'si
elti-ank

ipanimuleg
lukewinu
wa’so
keska’tu’
Tapuewei
Honour
wetapilo’tasiksip
Tapuewey
Tepknuset
amalkewaq
masgelmugsio
amalkasikel
niksamijewitki’k
'gta'pimuoq
apugjig
eymapWen
epipnasi
elikuwa'nu
'nqatmn
ekina’masiap
wetewinto'qNiskam
nekemowkewey
nemitu'n
kis-sutkis
ta'suguna'
pukwelipunqekl
Mekite'tmek
Pa’kewimk
kelu’sitaq
Pasena
“Mu
te’si’kw
ansale'wijik
“Ma
Sisua’qate’kn
“Mo
tua’lu
Jilapaquwikulten
pantetekip
unaqapem
L’nu’qamiksut
matnmuawt
apajat
Peju’k
qatayuk
newtuka’lukutiek
teli-anko’tmumk
kisa’matulti’tij
gesnugwateg
kisa’matulti’tip
gypsum
Api’s
Immersion
Cement
ji'nm
nasankewe’ktɨtew
lia’j
wla’lnu’kw
anqunkik
kespia'tuksi'
Pekije’k
apatkimuksin
mikwitetemitijel
malie’wiksipnaq
wtuisun
Ta’nkwiluoq
elukenuke
mawi-amskwesewe’l
poqji-newtituapnl
siawintoq
kejitu'tij
'gjigepmitelmate
Ika’q
Kisi’kipni’k
ewi’tmu
Ankaptmumk
metlasiska’q
nqamasa’lukwi’tij
sisku-iktuk
Asukomowey
masimk
tetuisik
niwianug
Munroe
‘Nujj
Jijaqemij
eweke
tel-tmsmumkl
mnte'muk
teli-melki-ikatmi’tij
Ewle’jit
keke

ankataqeney
knewite’lma
plamu’k
Tali-pkije’k
ewlitasik
wissukwa’tiji
Mtapsun
kepmi
saqmaw
telapukueykw
poqj
kepme
saqmaq
nigtuita'jig
poqt
they
ne’patue
ne’patuk
naqaligw
kniskamijinaqi’k
nugsaqatal
kwutanmina
telukupmite’lmuj
mawi-wlankuayanik
e'w'
piwa’kittaqne
tetuji­wula'matultie'k
wisukwa'tijik
naqtasikek
ejili
kina’matimk
esnoqnemk
alsutemaqeney
meto’temilit
Hugho’q
'Ms't
maqatkwik
Scare
jigs'tmu'tiwoq
anga
Mius
pemi-keknuika’tu’tij
t’es
wikultilijik
amgwesewei
'lsutmasg'pni
sike'wita'sultine
weleywioqipetli-n’tuanek
'ngutugwa'lugwa'
nutuwa'tiji
weji-nsituita’tim
etl-wutqutalusni’
iloqaptik
Lassui’pi’skwaq
pite’muji
ewlistuine
eteknuksip
gutang
wetnu’kwatkis
Kwetejki'
eykek
nemiskemun
keptinaq
ika’tu’n
kisikuiskw
wi’ka’tita
ignmugsi'gw
nasotutijel
wesua’toqsipne
netu’skasikip
Mi'kmamaq
ejaqjepulti’tij
musga'tugsitoqs'p
kisi-pisuney-iktuk
kast’pla
(tqamuewe'l
wsko’tmi
Vision
Ketloqo
tel-lukutisnik
kistliaqa
teluisap
amujpana
wjipqikayi’k
tle'litaqweskwijinultiji
weskwieliji
Sobeys

na’ji
MacIver
mimajuinu'l
Se’susewlite’liminami
mimajuinu'g
pema'tiji
we’jut
apoqntmu'
Mawio'miaq
Nekaw
tan-oc
nuttaq
maliamataq
wi’kiksip
wsitqamu
sikwoqasiknn
teli-alsumsultijik
nepitekem
emtoqwalg't
sewiska’tiklNa’kuset
apetapskenmenew
kelulk
Malsulian
glapis
ke’sk
tewo'laji
meki
wljewiaq
mselsit
wi’kikne
Eleke’wa’kik
ke’kupa’silin
elmayjituek
Pipanimk
alasutuo’kuom
nanipunke
Se'sus
lian
kwetej
Cornwallis
mteta'
ketlamsitasi
li-mawe-skuhutasu
nutajite’lsik
elisqapeka’tasik
Ta’n
maw-kespi-nujitewakitkisni’l
Maltaal
menipko’tasiksip
nemi’jik
Elmipunqekek
soqetuk
elàtuaj
Na'nike'
ungi'gu
stood
nisawtukwemkik
gisitugs'p
'gtlams'tm
weji-mno’tasikipn
kesatmu’tikip
ji’nmuk
mtoqalne
eskmatmi’titl
toqpie
etli-pukweliejik
Colleen
'Gt'tplutaqanminaq
ewe’wk
“Migwite'te'n
poqji-maliaptmekip
wktini
nemitu'titeg
Mable
pulja’ynewey
mikwaptikl
ewe’wt
klujjiewe
wikumiyo
mujik
wkoqwajo’taqann
Pem-pilua’sik
ne's
Smaliaewi'sgw
ne'w
Kespekewa
Kesi
tpia
Westaulg
etl-klu’si
aji-a’sikua’tijik
Nekmewey
lue'w

...

kno'tulu
kaqtiana’luksiwap
kis-tlim
nikanasit
nemia'tij
kisa’tumk
telo'lti'k
knijaninenuwaq
awlinu
Cree
Pacifique
alsusutl’l
pekije’nuke
Alasiéw-iktuk
natawa’qa’tekemkewey
nestm'k
tasnik
wenaqeken
penetkwipni
wejitasijik
Through
melkikna
kisa'tuek
waieknapie’
siawaskenasik
ala’tum
eykis
apkwa’lsi
eplewi-apankitmuk
kiseyasikw
ela'lukwe
mgo'tig
ugsmugwatew
nn'jan
gi'gamgo'nigtug
ki’
(Se'sus
puatmli
wanta’si
nikoq
ginualsin
ginualsia
elukwatasik
Nujintu’tijik
Kisimsntes
poqji-aluamkwia
eig
eim
siawa’tasiktn
“Gi'l
“Gi's
msiku’
minu’nsili
kewa’liki
assite’lmut
nemitu’kek
nisaskenasikel
alta'pni'
aklasiewi’sultine
kmu’ji’jl’
Carol
tkipowi
tli-apankituate
Inklinkewey
nuji-tlewistoq
(KAK)
amgweseweie
(kisitoqip
i’apu’tulti’tisnl
(Teluisi
ignmuates
Elue’ut
jitki'l
Aiseiao'q
lukwa’mit
Petkima'titl
ankateme
npo’qnek
uggistejumultiwe
wjisaqmaw
iknmuksi’tij
Project
iknmuksi’tis
pkesikn
Pointewey
telueioq
t
Amalkewaqn
ankaptmnew
wi'kupaltultim
mimejuinu’k
telawtiksip
tla’tekene
tapuwewey
tliapankitu

gle'pl
etuwiw
jugu'a'p
jipiskl
Morrell
nanipunqe
melkikna’tijik
eli’sman
ankweyu
wijitgweiwi
ne’ata’tew
ketantaqatie
musga's'g
ankuowe
utanjijk
mtmo’taqneye
Coast
weskaqeltimule
miawe’k
alayjimut
'gt'tplutaqanmua
waju’aqsip
un'jang
tleyin
un'jani
'gt'tplutaqanmuo
ugtatla'im
Polishaq
Mukla’qatikewey
kina’masin
pitkmalan
sqna’qwe’k
ugt'tplutaqanml
minigt'ta
nemito
mil
aji-kina’masultiji
kaqjesma’tiji
Kisu’lkw
Sante
sikalatkewultik
telui'tuluoq
nemitu
weskuma'titl
pekisitasij
Trai
ilajugtmi'tij
Poqji
kcitposultiyeq
Naqlaji
ila'kitu'nl
etloqsijig
Sqana’qw
wejiaqsipnl
usitkamu
ul'te'tutew
igtigig
ut-jijaqmijuwaq
matnew
usgupn
kaqa’sit
semaqita’jik
ksite’l-muksi
matney
tua’laten
kisi-un-smk-
alankua’ti’tij
gisteju'g
telitekemk
ika’te
asoqma’timk
ika’ti
kampni’l
iknemuatis
naji-mittukwaltiji
kaqikisa’tasiksip
tepi-pkije’nuk
Kejitu’n
nsaqamamine
Unama’kiey
camsoq
metui’plna’tijekKekkam
enqa’sit
Gaiapaseg
nepu'tijig
apaja’luksine
Pun-kina’muejek
pejiapu’p
Islands
enqa’sin
Istue’k
Teli-nqase’k
S

nastekl
sma'gnisg
Keji’jik
mittukutitewe’k
Eskisoqni
gesatmul
te’sipunqe
wije'wa'titl
wigumut'pnig
telui'tapn
Kejituek
kiskaja’tu’nl
ejikla’tu
peitapenik
“Wije'wi
nespikima'tiji
(peaked
eltoq
Saqewel
wejiapu’kwetumkis
wowjiktuk
nikmanaq
ignmuiweg
nsamqultik
Se’susal
nujeyaqik
maw-nutqwe’te’w
teli-o’plsitmi’tij
Naji
“Mukk
wla’matultiji
ut-makmikewmuwaq
teluetl
smtug
Kina’matnewo’kuo’ml
pem-wije’tultij
tmasqite’kl
poqji-kina’mua’tijek
nestmu'tiwoq
Brazi
ugsua'la'titl
welapetmu'tioq
apoqnmat
Pemkopiek
na'ngl
Menaq
ilamko’tasikl
“Wutankewa’j
Malgu
kisima’muio’pla’lul
wel-nenuatl
wesua'la'titl
iknmuipni’k
teliaqewe
E’pitewey
oqonoqopisa’sitKe’sk
kitoqo
Mawiespe'k
kitoqi
ul'ta'sultigoq
mawiómíl
panto’tumkis
Sqna’qwey
nesipunqekl
litaqnwi’kikemkey
ki'katkatultiti
iga'tutew
wl-nenasi
iganpugutuaji
iveup'owu
astua'li
pikaqn
alaqtekewinu’kKi’l
nikantoq
anko'tmu'k
nepapigwa'iap
kina’mues
kina’muet
Wunjek
kepmite'k
pemlo’tatl
kis-kina’muaj
nujiempitekejit
Granta
ankama’tiji
KAK
pejiwsultiji
Kasa't

amalintemkewe
utini
nasqoqalaji
wisesk
Kujjinaqi’k
kiwnaqaj
Cla
Nevin
ela’lis
temkwalikenejk
telatutijel
mijua’ji’jNtininenaq
mile'sit
elege'wit
toqi’katisn
ne’w
ignmatmnew
wigulti'gw
eimn
paqalaiwaqann
Pa’skite’kem
pipanimtultiji
gepmitelmu
jijklue’wjua’pi’
eytenukel
gepmitelmi
pukwelk
lie
Nin
ntkelutmalsewie
wenujk
alasutuo’kuoml
ilta'tuaji
pektaqikwatasi
Nip
ketanujik
nemetkekewaq
[Etna
lo'q
welpilitl
Head
kaqi-ika’lsis
wasoqa’tasik
piskweta’tij
iknmuwatl
pisgwa't
'Gsalioq
nestuatl
Maupltu
[Westaulg
pisgwa'n
pisgwa'j
Arsenaul
gi'le
lasui’paqi’k
ektapekiaqen
aknutmuatl
teluitmi'tij
nekmowey
We’koqma’
kaqteskasi
telipunqek
klo'tesnu
pipnaqan
jugu'ewasn
elukewa’tijik
kelu’k
ekatu
Nkma'ji'j
Mattaqte’kney–iktuk
x
we'jia'tikaq
nipke
pusultijik
Morrison
wnaqa'tumk
kis-pukuspenu
E'pijik
elokwistasik
nuta’ji
jiklijuitew
pata'taqatiji
isop
wape'litl
kisitasiksipn
ipunke
leskikenewik
se'sapaqte'g
Mtul’katlku
mijisit
ujjuaqi'g
westatew
musga'toq
kaskiktlnaqenipunekek
awti
waltes
ankamatiji
meke

Maw-lukutimk
essawiatk
'mtesgmaq
laqana’tasikWktini
kinamasimkewokuomk
siawa’toqsip
memkewey
keknuatekekel
kisi-iloqaptmi’tij
ne’slijik
elapilan
attelma
mijuaji’j
attelmi
Gunn
kepjoqa’tasiksip
kiwalsultiji
kaqjesma'tiji
klotesnuk
nkamilamunkaniapsuti
kukenmenej
Exercise
elta'ji
na’tuen
aklasiewe’kati
peju
Sésú
peji
Apektek
atkitemuimsultikAq
tel-kina’masim
ukji
welkne’kk
mimajultine
matneke
gujjiewto't
asuwita'ql
Pau
“Nu'
teleyn
amalkatemanek
nemia'tite
jiloltikwik
Eymu’tijik
nemia'titl
mtapsunn
sisi
waqamo'ltioq
náté
(Margaret
sist
ut-utanmuaq
etawaqtemat
getlams'tmu'tiwoq
Gli'gewei
nestasultijig
ketlamsitas
ektapekiaqnn
Donald
elpukua’timk
kita
amasiko'lti'k
jijaqmijuik-awti
wisuoq
kitk
petasisnl
Wejueya
Ktaqamku
wajuwaq
Mos
Moq
wli-nenatultine
ksi
kisikui’skwa
Lillian
pilui'sit
tela’tu’ti
jukwa’tine
wutj
wenaqamikekl
alkitoqetaqiatiji
Panuijkatasik
iapjiwewe
apu’kwetumk
eliu
kinua’tuatl
telita’simk
assutm
kenatutijel
geggun
(kisna
gnu'tmuaji
wecihc
te’sik
weji-ajkno’ltijik
nujo’tkl


In [17]:
mi_dictionary.export('/home/apatra/fastText/fastText_multilingual-master/micmaq8.vec')